In [ ]:
import pandas as pd
import sqlite3
! pip install yfinance
! pip install quandl
import quandl as quandl
from datetime import datetime
import pprint as pp
import yfinance as yf






Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# backtest inputs
bt_inputs = {'tickers': ['HLT', 'UAL', 'CCL', 'DAL', 'MAR'],
'start_date': '2017-01-01',
'end_date': '2023-05-01'}

#start = bt_inputs['start_date']
#end = bt_inputs['end_date']
# Create a database to store the stock data

# create a sql connection
con = sqlite3.connect('stocks.db')
c = con.cursor()
# create price table
query1 = """CREATE TABLE IF NOT EXISTS prices (
Date TEXT NOT NULL,
ticker TEXT NOT NULL,
price REAL,
PRIMARY KEY(Date, ticker)
)"""
c.execute(query1.replace('\n',' '))
# create volume table
query2 = """CREATE TABLE IF NOT EXISTS volume (
Date TEXT NOT NULL,
ticker TEXT NOT NULL,
volume REAL,
PRIMARY KEY(Date, ticker)
)"""
c.execute(query2.replace('\n',' '))

# inputs
select_tickers = bt_inputs['tickers']
start_date = bt_inputs['start_date']
end_date = bt_inputs['end_date']

# construct query
query = """
select * from prices
where ticker in ('"""+ "','".join(select_tickers) + """')
and Date >= '"""+ start_date + """'
and Date < '""" + end_date + "'"
c.execute(query.replace('\n',' '))
result = pd.DataFrame(c.fetchall(), columns = ['Date', 'ticker', 'price'])
# convert to datetime
result['Date'] = pd.to_datetime(result['Date'])

pp.pprint(result.head())

        Date ticker      price
0 2017-01-03    CCL  46.789989
1 2017-01-03    DAL  45.714943
2 2017-01-03    HLT  54.502747
3 2017-01-03    MAR  78.129959
4 2017-01-03    UAL  72.709999


In [ ]:
# Download stock data

def download(bt_inputs, proxy = None):
 #    data = quandl.get('NSE/OIL', start_date = start,
#                 end_date = end,
#                 authtoken = 'XfxsCYJQVpKbCcQRi3d8')

      data2 = yf.download(tickers = bt_inputs['tickers'],
                        start = bt_inputs['start_date'],
                        end = bt_inputs['end_date'],
                        interval ='1d',
                        prepost = True,
                        threads = True,
                        proxy = proxy
                        )
      return (data2)


# test method
test2 = download(bt_inputs)
#pp.pprint(test)
pp.pprint(test2)

test2.to_csv('data.csv')


[*********************100%***********************]  5 of 5 completed
            Adj Close                                                \
                  CCL        DAL         HLT         MAR        UAL   
Date                                                                  
2017-01-03  46.789993  45.714943   54.502750   78.129967  72.709999   
2017-01-04  47.400219  46.842106   56.244102   77.978439  73.019997   
2017-01-05  48.198895  45.945923   55.584686   77.448090  70.930000   
2017-01-06  48.145046  45.899723   55.730152   77.940567  71.370003   
2017-01-09  48.180939  45.908966   55.235588   77.893219  71.279999   
...               ...        ...         ...         ...        ...   
2023-04-24   9.280000  34.119999  148.295578  173.791214  43.930000   
2023-04-25   8.960000  33.110001  146.177780  172.325760  42.490002   
2023-04-26   8.760000  32.750000  141.192978  164.609665  42.099998   
2023-04-27   8.900000  33.330002  141.592560  164.878845  42.830002   
2023-04-

In [ ]:
# isolate the adjusted prices and volume data.

# Adjusted prices are useful as they are adjusted for corporate actions such as dividend payments and stock splits.

adj_close = test2['Adj Close']
volume = test2['Volume']

# convert wide to long
adj_close_long = pd.melt(adj_close.reset_index(), id_vars='Date', value_vars=bt_inputs['tickers'], var_name ="ticker", value_name="price")
volume_long = pd.melt(volume.reset_index(), id_vars='Date', value_vars=bt_inputs['tickers'], var_name = "ticker", value_name = "volume")
pp.pprint (adj_close_long)
pp.pprint (volume_long)
#Push financial data into database
# adj_close_long.to_sql('prices', con, if_exists='append', index=False)
# volume_long.to_sql('volume', con, if_exists='append', index=False)

#Pull financial data from the database
# inputs
select_tickers = bt_inputs['tickers']
start_date = bt_inputs['start_date']
end_date = bt_inputs['end_date']

# construct query
query = """
select * from prices
where ticker in ('"""+ "','".join(select_tickers) + """')
and Date >= '"""+ start_date + """'
and Date < '""" + end_date + "'"
c.execute(query.replace('\n',' '))
result = pd.DataFrame(c.fetchall(), columns = ['Date', 'ticker', 'price'])
# convert to datetime
result['Date'] = pd.to_datetime(result['Date'])
result.to_csv('data_organized.csv')

           Date ticker       price
0    2017-01-03    HLT   54.502750
1    2017-01-04    HLT   56.244102
2    2017-01-05    HLT   55.584686
3    2017-01-06    HLT   55.730152
4    2017-01-09    HLT   55.235588
...         ...    ...         ...
7950 2023-04-24    MAR  173.791214
7951 2023-04-25    MAR  172.325760
7952 2023-04-26    MAR  164.609665
7953 2023-04-27    MAR  164.878845
7954 2023-04-28    MAR  168.816635

[7955 rows x 3 columns]
           Date ticker   volume
0    2017-01-03    HLT  3328021
1    2017-01-04    HLT  5614500
2    2017-01-05    HLT  5122200
3    2017-01-06    HLT  3359000
4    2017-01-09    HLT  3921800
...         ...    ...      ...
7950 2023-04-24    MAR  1146800
7951 2023-04-25    MAR  2226300
7952 2023-04-26    MAR  3087400
7953 2023-04-27    MAR  2105900
7954 2023-04-28    MAR  1645900

[7955 rows x 3 columns]
